In [22]:
import numpy as np
import pandas as pd

In [5]:
DATA = '../../data'

In [8]:
def read_stocks():
    stock_df = pd.read_csv(f'{DATA}/stockuniverse.csv')
    stocks = list(stock_df.stock)
    return stocks

In [10]:
stocks = read_stocks()

In [57]:
class Gene:
    def __init__(self, stocks):
        '''
        Args:
        - stocks: either:
          - dict: {'symbol', <pct_of_portfolio>} OR
          - list: ['symbol_1', ..., 'symbol_n'] (pct will be even 
              among all)
        '''
        self.TOLERANCE = 0.00000001
        if type(stocks) is list:
            stocks = self._init_stocks_from_list(stocks)
        if isinstance(stocks, dict):
            total = sum(stocks.values())
            assert abs(total - 1) < self.TOLERANCE,\
                f'Stock percentages must sum to 1 (got {total})'
        else:
            raise ValueError('stocks must be a <list> or <dict>')
        self.stocks = stocks
        
    @staticmethod
    def _init_stocks_from_list(stocks):
        n = len(stocks)
        stocks = {stock: 1/n for stock in stocks}
        return stocks
    
    def __str__(self):
        out = ''
        for stock, pct in self.stocks.items():
            if pct > self.TOLERANCE:
                out += f'{stock:5s}: {pct:.5f}\n'
        return out
    
    def mutate(self, sd):
        for stock, pct in self.stocks.items():
            pct += np.random.normal(scale=sd)
            pct = max(pct, 0)
            self.stocks[stock] = pct
        self.stocks = self._normalize(self.stocks)
        
    def _normalize(self, stocks):
        total = sum(stocks.values())
        for stock, pct in stocks.items():
            stocks[stock] = pct / total
        return stocks
            
    def reproduce(self, other):
        child = {}
        for stock in self.stocks:
            which = np.random.choice([self, other], 1)[0]
            val = which.stocks.get(stock, 0)
            child[stock] = val
        child = self._normalize(child)
        child = Gene(child)
        return child

In [58]:
gene1 = Gene(stocks)
gene1.mutate(0.2)
#print(gene1)

gene2 = Gene(['abmd', 'bili'])
#print(gene2)

In [59]:
child = gene1.reproduce(gene2)
print(child)

abmd : 0.06896
atvi : 0.11032
axon : 0.02662
bili : 0.44858
bynd : 0.06743
fsly : 0.00378
gh   : 0.02588
lulu : 0.05832
meli : 0.01063
pton : 0.02889
se   : 0.03444
team : 0.07114
twtr : 0.04499

